In [ ]:
engine = create_engine(connection_string)
from sqlalchemy import MetaData, Table

metadata = MetaData()
issues_table = Table("Issues", metadata, autoload_with=engine)

with engine.connect() as connection:
    print("Kết nối thành công!")
    query = select(issues_table)
    result = connection.execute(query)
    for row in result:
        print(row)
    for column in issues_table.columns:
        print(column.name)

/tmp/ipykernel_10574/2623609048.py:13: SAWarning: Unrecognized server version info '17.0.800.3'.  Some SQL Server features may not function properly.
  issues_table = Table("Issues", metadata, autoload_with=engine)


Kết nối thành công!
id
service_name
description
mcp_url


In [16]:
# Define a function that the model can call to control smart lights
set_light_values_declaration = {
    "name": "set_light_values",
    "description": "Sets the brightness and color temperature of a light.",
    "parameters": {
        "type": "object",
        "properties": {
            "brightness": {
                "type": "integer",
                "description": "Light level from 0 to 100. Zero is off and 100 is full brightness",
            },
            "color_temp": {
                "type": "string",
                "enum": ["daylight", "cool", "warm"],
                "description": "Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.",
            },
        },
        "required": ["brightness", "color_temp"],
    },
}

# This is the actual function that would be called based on the model's suggestion
def set_light_values(brightness: int, color_temp: str) -> dict[str, int | str]:
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {"brightness": brightness, "colorTemperature": color_temp}

In [64]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from app.core.config import settings

from google import genai
from google.genai import types

tools = types.Tool(function_declarations=[set_light_values_declaration])
config = types.GenerateContentConfig(tools=[tools])

client = genai.Client(api_key=settings.GEMINI_API_KEY)
chat = client.chats.create(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        system_instruction="Bạn là một trợ lý ảo thông minh, có thể giúp tôi điều khiển đèn thông minh trong nhà.",
        temperature=0.0,
        tools=[tools],
    ),
    history=[
        types.Content(role='user', parts=[{"text": "Xin chào"}]),
        types.Content(role='model', parts=[{"text": "Xin chào! Tôi có thể giúp gì cho bạn hôm nay?"}])
    ]
)

In [65]:
response = chat.send_message("Tôi muốn bật đèn phòng khách với độ sáng 80 và màu ban ngày.")

In [71]:
response.function_calls[0].args

{'brightness': 80, 'color_temp': 'daylight'}

In [67]:
response.text

In [48]:
for message in chat.get_history():
    print(f'role - {message.role}',end=": ")
    print(message.parts[0].text)

role - user: Xin chào
role - model: Xin chào! Tôi có thể giúp gì cho bạn hôm nay?
role - user: bạn làm ơn bật đèn sáng 100% và chuyển sang chế độ ánh sáng ban ngày
role - model: None


In [ ]:
chat_history = [{'role': 'user', 'content': 'Xin chào'},
                {'role': 'model', 'content': 'Xin chào! Tôi có thể giúp gì cho bạn hôm nay?'}]

history = []
for message in chat_history:
    

role
content
role
content
